In [1]:
# Import
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, SimpleRNN, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD,Adam

In [ ]:
# parsing data
my_data = np.genfromtxt('games_data.csv', delimiter=',') #read data
my_data = np.asarray(my_data[1:]) #ignore header

target = my_data[:,0].astype(int) #first column is target, blueWins = 1
features = my_data[:,1:]

# breakup data
x, x_test, y, y_test = train_test_split(features, target, test_size=0.2) # 20% test
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.25) # 25% * 80% = 20% validation

N, D = x_train.shape

# scale the data
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)
x_test = scaler.transform(x_test)

In [ ]:
# hyper parameters
alpha   = [0.001, 0.0025, 0.01, 0.025]      # learning rate
MaxIter = 50        # Maximum iteration
nodes = [16,32,64,128]      # Hidden layer node count

In [ ]:
# Base model
# If blue have more kills then 1, else 0
# Tie break by comparing assists 
true_pred = 0
total_pred = len(y_test)
for i in range(len(x_test)):
  pred = 0
  if x_test[i][3] > x_test[i][22]:
    pred = 1
  elif x_test[i][5] > x_test[i][24]:
    pred = 1
  if pred == y_test[i]:
    true_pred += 1
print("base model test_accuracy:", true_pred/total_pred)


base model test_accuracy: 0.6958502024291497


In [ ]:
#### Models ####
# The following three sections are for the models
# Only run the wanted model and move on to training section

In [ ]:
# Linear Regression
i = Input(shape=(D,))
x = Dense(1, activation="linear")(i)

model = Model(i, x)

# Toggle Nodes in hyperparameters selection
isNN = False

In [ ]:
# Logistic Regression
i = Input(shape=(D,))
x = Dense(1, activation="sigmoid")(i)

model = Model(i, x)
# Toggle Nodes in hyperparameters selection
isNN = False

In [ ]:
# Neural Network
def getNNmodel(node):
  i = Input(shape=(D,))
  x = Dense(node, activation="linear")(i)
  x = Dense(1, activation="sigmoid")(x)

  return Model(i, x)

# Toggle Nodes in hyperparameters selection
isNN = True

In [ ]:
#### End of Models ####

In [ ]:
# Train & Evaluate Model
accuracy_best = 0
model_best = None
a_best = 0
node_best = 0
history_best = None

if isNN==False: 
  # grid search best alpha
  for a in alpha:
    opt = tf.keras.optimizers.SGD(learning_rate=a)
    model.compile(optimizer='SGD', loss='binary_crossentropy', metrics=['accuracy'])
    # Train the model
    history = model.fit(x_train, y_train, verbose=0, validation_data=(x_val, y_val), epochs=MaxIter)
    # Evaluate the model by accuracy
    accuracy = model.evaluate(x_val, y_val)[1]
    # Select best model
    if accuracy >= accuracy_best:
      accuracy_best = accuracy
      model_best = model
      a_best = a
      history_best = history

if isNN==True:
  # grid search best alpha
  for a in alpha:
    # grid search best node count
    for node in nodes:
      model = getNNmodel(node)

      opt = tf.keras.optimizers.SGD(learning_rate=a)
      model.compile(optimizer='SGD', loss='binary_crossentropy', metrics=['accuracy'])
      # Train the model
      history = model.fit(x_train, y_train, verbose=0, validation_data=(x_val, y_val), epochs=MaxIter)
      # Evaluate the model by accuracy
      accuracy = model.evaluate(x_val, y_val)[1]
      # Select best model
      if accuracy >= accuracy_best:
        accuracy_best = accuracy
        model_best = model
        a_best = a
        node_best = node
        history_best = history

# Test final model performance
print("Test score:", model_best.evaluate(x_test, y_test))
print("accuracy_best:", accuracy_best)
print("a_best:", a_best)
print("node_best:", node_best)

In [ ]:
# plot to see result
import matplotlib.pyplot as plt
plt.plot(history_best.history['loss'], label='loss')
plt.plot(history_best.history['val_loss'], label='val_loss')
plt.legend()

In [ ]:
plt.plot(history_best.history['accuracy'], label='acc')
plt.plot(history_best.history['val_accuracy'], label='val_acc')
plt.legend()